In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

# Set paths to your dataset folders
train_dir = '/home/esidserver/datasets/testing_cls/training-v3/train/'
val_dir = '/home/esidserver/datasets/testing_cls/training-v3/val/'
test_dir = '/home/esidserver/datasets/testing_cls/training-v3/test/'

# Image data generator with data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Data generator for validation and test
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Data loaders
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Load the VGG19 model with pre-trained weights
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom top layers for classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Define early stopping and model checkpoint
early_stopping = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('vgg19_best_model.keras', save_best_only=True, monitor='val_loss')

# Train the model
history = model.fit(
    train_generator,
    epochs=250,
    validation_data=val_generator,
    callbacks=[early_stopping, model_checkpoint]
)


2024-09-19 20:55:19.408600: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-19 20:55:19.433625: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-19 20:55:19.466853: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-19 20:55:19.476340: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-19 20:55:19.500160: I tensorflow/core/platform/cpu_feature_guar

Found 5195 images belonging to 4 classes.
Found 1482 images belonging to 4 classes.
Found 740 images belonging to 4 classes.


2024-09-19 20:55:22.002080: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22181 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:af:00.0, compute capability: 8.6


Epoch 1/250


/home/esidserver/miniforge3/envs/tf/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1726754129.454643 4007265 service.cc:146] XLA service 0x79216c003bd0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1726754129.454675 4007265 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-09-19 20:55:29.496271: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-09-19 20:55:29.851936: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8902
2

...

 34/163 ━━━━━━━━━━━━━━━━━━━━ 47s 368ms/step - accuracy: 0.3142 - loss: 1.4029

2024-09-19 20:55:49.197096: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_944', 12 bytes spill stores, 12 bytes spill loads



162/163 ━━━━━━━━━━━━━━━━━━━━ 0s 379ms/step - accuracy: 0.4243 - loss: 1.3129

2024-09-19 20:56:43.981648: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_252', 12 bytes spill stores, 12 bytes spill loads



163/163 ━━━━━━━━━━━━━━━━━━━━ 82s 446ms/step - accuracy: 0.4254 - loss: 1.3118 - val_accuracy: 0.5607 - val_loss: 1.1022
Epoch 2/250
163/163 ━━━━━━━━━━━━━━━━━━━━ 64s 376ms/step - accuracy: 0.6252 - loss: 1.0437 - val_accuracy: 0.6538 - val_loss: 0.9687
Epoch 3/250
163/163 ━━━━━━━━━━━━━━━━━━━━ 62s 367ms/step - accuracy: 0.6830 - loss: 0.9182 - val_accuracy: 0.6775 - val_loss: 0.8888
Epoch 4/250
163/163 ━━━━━━━━━━━━━━━━━━━━ 64s 373ms/step - accuracy: 0.7131 - loss: 0.8212 - val_accuracy: 0.6889 - val_loss: 0.8350
Epoch 5/250
163/163 ━━━━━━━━━━━━━━━━━━━━ 63s 372ms/step - accuracy: 0.7346 - loss: 0.7542 - val_accuracy: 0.6957 - val_loss: 0.7955
Epoch 6/250
163/163 ━━━━━━━━━━━━━━━━━━━━ 62s 364ms/step - accuracy: 0.7397 - loss: 0.7310 - val_accuracy: 0.7072 - val_loss: 0.7617
Epoch 7/250
163/163 ━━━━━━━━━━━━━━━━━━━━ 59s 344ms/step - accuracy: 0.7640 - loss: 0.6629 - val_accuracy: 0.7099 - val_loss: 0.7519
Epoch 8/250
163/163 ━━━━━━━━━━━━━━━━━━━━ 59s 345ms/step - accuracy: 0.7841 - loss: 0.617

In [2]:
# Fine-tuning: Unfreeze some layers of the base model
for layer in base_model.layers:
    layer.trainable = True

# Compile the model again
model.compile(optimizer=Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

# Continue training with fine-tuning for 100 epochs
history_finetune = model.fit(
    train_generator,
    epochs=100,  # Fine-tuning for 100 epochs
    validation_data=val_generator,
    callbacks=[early_stopping, model_checkpoint]
)

Epoch 1/100


2024-09-19 22:54:06.247073: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1654', 76 bytes spill stores, 76 bytes spill loads



163/163 ━━━━━━━━━━━━━━━━━━━━ 94s 421ms/step - accuracy: 0.8962 - loss: 0.2887 - val_accuracy: 0.8084 - val_loss: 0.5737
Epoch 2/100
163/163 ━━━━━━━━━━━━━━━━━━━━ 60s 351ms/step - accuracy: 0.9446 - loss: 0.1432 - val_accuracy: 0.8381 - val_loss: 0.4739
Epoch 3/100
163/163 ━━━━━━━━━━━━━━━━━━━━ 58s 342ms/step - accuracy: 0.9582 - loss: 0.1250 - val_accuracy: 0.8394 - val_loss: 0.5110
Epoch 4/100
163/163 ━━━━━━━━━━━━━━━━━━━━ 60s 351ms/step - accuracy: 0.9742 - loss: 0.0719 - val_accuracy: 0.8650 - val_loss: 0.4141
Epoch 5/100
163/163 ━━━━━━━━━━━━━━━━━━━━ 59s 345ms/step - accuracy: 0.9792 - loss: 0.0561 - val_accuracy: 0.8657 - val_loss: 0.4787
Epoch 6/100
163/163 ━━━━━━━━━━━━━━━━━━━━ 59s 347ms/step - accuracy: 0.9808 - loss: 0.0484 - val_accuracy: 0.8866 - val_loss: 0.4033
Epoch 7/100
163/163 ━━━━━━━━━━━━━━━━━━━━ 58s 342ms/step - accuracy: 0.9887 - loss: 0.0350 - val_accuracy: 0.8354 - val_loss: 0.6180
Epoch 8/100
163/163 ━━━━━━━━━━━━━━━━━━━━ 58s 338ms/step - accuracy: 0.9864 - loss: 0.042

In [3]:
# Evaluate on the test set
test_loss, test_acc = model.evaluate(test_generator)

# Calculate final training and validation accuracy
train_acc = history_finetune.history['accuracy'][-1] if 'accuracy' in history_finetune.history else None
val_acc = history_finetune.history['val_accuracy'][-1] if 'val_accuracy' in history_finetune.history else None

# Print the results
print(f"Final Training Accuracy: {train_acc:.4f}")
print(f"Final Validation Accuracy: {val_acc:.4f}")
print(f"Test Accuracy: {test_acc:.4f}, Test Loss: {test_loss:.4f}")

# Save the final model
model.save('vgg19_final_model.keras')

23/24 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - accuracy: 0.9152 - loss: 0.3762

2024-09-20 00:29:56.370219: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_252', 4 bytes spill stores, 4 bytes spill loads



24/24 ━━━━━━━━━━━━━━━━━━━━ 5s 210ms/step - accuracy: 0.9152 - loss: 0.3760
Final Training Accuracy: 0.9988
Final Validation Accuracy: 0.9339
Test Accuracy: 0.9149, Test Loss: 0.3730
